## DATA AND TABLE MODELING

Note: The following variables are part of the dataset structure, but all corresponding cells and operations have been intentionally removed or cancelled due to the confidential nature of the data

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import (
    col,
    when,
    dayofmonth,
    month,
    year,
    dayofweek,
    sum    as spark_sum,
    max    as spark_max,
    avg    as spark_avg,
    row_number,
    coalesce
)
from pyspark.sql.window import Window

In [0]:
# 1. Chargement des tables Spark
items            = spark.table("items_state")
orders           = spark.table("order")
sales_store      = spark.table("store")
products         = spark.table("product")
product_variants = spark.table("variant")

# 2. Lecture du CSV local en pandas puis conversion en Spark DataFrame
import pandas as pd
local_path       = "Recap 30-09-2024(Feuil1).csv"
product_info_pd  = pd.read_csv(local_path, sep=";", encoding="latin1")
product_info_pd  = product_info_pd.rename(columns={"code - Copier": "Code Gold"})
product_info     = spark.createDataFrame(product_info_pd)

# 3. Nettoyage générique de la colonne 'code' : on prend tout ce qui suit le dernier '-'
from pyspark.sql.functions import substring_index, col

products = products.withColumn(
    "code_cleaned",
    substring_index(col("code"), "-", -1)
)

# 4. Enchaînement des jointures
df_join = (
    items
    .join(product_variants, items["variant_id"] == product_variants["id"])
    .join(orders,          col("order_number") == col("number"))
    .join(sales_store,     orders["store_id"] == sales_store["id"], how="left")
    .join(products,        product_variants["product_id"] == products["id"], how="left")
    .join(
        product_info,
        products["code_cleaned"] == product_info["CODE_INT"],
        how="left"
    )
)



In [0]:
# Préparation des données
final_df = df_join.select(
    "variant_id",
    "lstore_id",
    "Order_Number",
    "quantity_delivery",
    "quantity_sales",
    "STATUS",
    "checkout_completed_at",
    "state",
    "LIB_ETAT_ARTICLE",
    "GRP",
    "DEP",
    "RAY",
    "SAISONNALITE_ARTICLE"
)




com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
# Ajout de la colonne stockout
final_df = final_df.withColumn(
    "stockout",
    when(col("STATUS") == "unfound", 1).otherwise(0)
)


In [0]:
# Si ta colonne magasin s’appelle simplement 'store_id' :
df_store25 = final_df.filter(col("store_id") == 25)



com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
from pyspark.sql.functions import col, to_date, lit

# ------------------------------------------------------------------
# 1) Indique tes dates ici ⬇︎ (format AAAA-MM-JJ)
# ------------------------------------------------------------------
date_debut = "2024-01-01"      # ← remplace par ta date de début
date_fin   = "2025-07-20"      # ← remplace par ta date de fin
# ------------------------------------------------------------------

# 2) Conversion en type date pour éviter les surprises
date_debut = to_date(lit(date_debut))
date_fin   = to_date(lit(date_fin))

# 3) Filtre
df_filtre = df_store25.filter(
    (col("checkout_completed_at").cast("date") >= date_debut) &
    (col("checkout_completed_at").cast("date") <= date_fin)
)




---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-8890019767672582>, line 15
     12 date_fin   = to_date(lit(date_fin))
     14 # 3) Filtre
---> 15 df_filtre = df_store25.filter(
     16     (col("checkout_completed_at").cast("date") >= date_debut) &
     17     (col("checkout_completed_at").cast("date") <= date_fin)
     18 )

NameError: name 'df_store25' is not defined

In [0]:
# ⚠️ 1. Active Arrow (copie en mémoire plus rapide)
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

# 2. Si ton DataFrame s’appelle daily_data :
pdf = df_store25.toPandas()      # PySpark classique
# pdf = daily_data.to_pandas()   # Spark Connect (nouvelle API)

# 3. Vérifie
print(pdf.shape)
pdf.head()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6620639893672549>, line 2
      1 # ⚠️ 1. Active Arrow (copie en mémoire plus rapide)
----> 2 spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
      4 # 2. Si ton DataFrame s’appelle daily_data :
      5 pdf = df_store25.toPandas()      # PySpark classique

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/conf.py:46, in RuntimeConf.set(self, key, value)
     44 op_set = proto.ConfigRequest.Set(pairs=[proto.KeyValue(key=key, value=value)])
     45 operation = proto.ConfigRequest.Operation(set=op_set)
---> 46 result = self._client.config(operation)
     47 for warn in result.warnings:
     48     warnings.warn(warn)

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1858, in SparkConnectClient.config(self, operation)
   1856     raise SparkConne